In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from sklearn.metrics import mean_squared_error

In [2]:
import xgboost as xgb

In [3]:
xgb.__version__

'1.1.1'

In [4]:
DATA_DIR_PATH = os.path.join('..', '..', 'data')

def load_data(file_name):
  file_path = os.path.join(DATA_DIR_PATH, file_name)
  return pd.read_csv(file_path, header=0)

In [5]:
df_train = load_data('bookmark_train_data.csv')
print(len(df_train))
#df_train.head(3)

199


In [6]:
from sklearn.preprocessing import LabelEncoder

cat_le = LabelEncoder()
cat_le.fit(df_train['category'])

WORD_CLASSE_LABELS = ['その他', 'フィラー', '副詞', '助動詞', '助詞',
                     '動詞', '名詞', '形容詞', '感動詞', '接続詞',
                     '接頭詞', '記号', '連体詞',]
new_wc_columns = {}
for (i, name) in enumerate(WORD_CLASSE_LABELS):
    new_wc_columns['wc_sum_{}'.format(name)] = 'wc_sum_{}'.format(i)
    new_wc_columns['wc_rate_{}'.format(name)] = 'wc_rate_{}'.format(i)


def data_preprocess(df):
    new_df = df.drop(['ncode', 'title', 'bookmark_cat'], axis=1)

    new_df['category'] = cat_le.transform(new_df['category'])

    new_df['rating'] = new_df['rating'] * 10
    new_df = new_df.astype({'rating': 'int32'})

    #wc_columns = [col for col in new_df.columns if col.find('wc_') != -1]
    #new_df = new_df.drop(wc_columns, axis=1)
    wc_columns = [col for col in new_df.columns if col.find('wc_sum_') != -1]
    new_df = new_df.drop(wc_columns, axis=1)
    new_df = new_df.rename(columns=new_wc_columns)

    kw_columns = [col for col in new_df.columns if col.find('kw_') != -1]
    new_df = new_df.drop(kw_columns, axis=1)

    return new_df

In [7]:
df_train.keys()

Index(['ncode', 'title', 'category', 'bookmark_cat', 'created_at',
       'updated_at', 'char_count_sum', 'new_line_count_sum',
       'talk_char_count_sum', 'new_line_count_rate_sum',
       'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg',
       'talk_char_count_avg', 'new_line_count_rate_avg',
       'talk_char_count_rate_avg', 'wc_sum_その他', 'wc_sum_フィラー', 'wc_sum_副詞',
       'wc_sum_助動詞', 'wc_sum_助詞', 'wc_sum_動詞', 'wc_sum_名詞', 'wc_sum_形容詞',
       'wc_sum_感動詞', 'wc_sum_接続詞', 'wc_sum_接頭詞', 'wc_sum_記号', 'wc_sum_連体詞',
       'wc_rate_その他', 'wc_rate_フィラー', 'wc_rate_副詞', 'wc_rate_助動詞',
       'wc_rate_助詞', 'wc_rate_動詞', 'wc_rate_名詞', 'wc_rate_形容詞', 'wc_rate_感動詞',
       'wc_rate_接続詞', 'wc_rate_接頭詞', 'wc_rate_記号', 'wc_rate_連体詞', 'kw_冒険者',
       'kw_魔王', 'kw_日常', 'kw_成り上がり', 'kw_内政', 'kw_恋愛', 'kw_無双', 'kw_青春',
       'kw_ハーレム', 'kw_男主人公', 'kw_奴隷', 'kw_テンプレ', 'kw_勘違い', 'kw_ほのぼの', 'kw_貴族',
       'kw_ギルド', 'kw_コメディ', 'kw_異世界', 'kw_エルフ', 'kw_ファンタジー', 'kw_オリジナル戦記',
       

In [8]:
df_train2 = data_preprocess(df_train)
df_train2.head(3)

,category,created_at,updated_at,char_count_sum,new_line_count_sum,talk_char_count_sum,new_line_count_rate_sum,talk_char_count_rate_sum,char_count_avg,new_line_count_avg,...,wc_rate_4,wc_rate_5,wc_rate_6,wc_rate_7,wc_rate_8,wc_rate_9,wc_rate_10,wc_rate_11,wc_rate_12,rating
0,6,1.586338e+09,1.593468e+09,92340,3840,28950,0.041585,0.313515,3078.0,128.0,...,0.237664,0.226083,0.225076,0.013595,0.006042,0.007049,0.006042,0.144008,0.011078,30
1,6,1.590714e+09,1.593338e+09,77880,4140,4950,0.053159,0.063559,2596.0,138.0,...,0.238590,0.238013,0.217215,0.017909,0.001155,0.010399,0.001155,0.138070,0.013287,40
2,6,1.575176e+09,1.593429e+09,59880,3090,10230,0.051603,0.170842,1996.0,103.0,...,0.240486,0.177328,0.285020,0.014575,0.000000,0.005668,0.008907,0.148178,0.005668,30


In [9]:
X = df_train2.drop(['rating'], axis=1).values
y = df_train2['rating'].values
print(X.shape, y.shape)

(199, 26) (199,)


In [10]:
feature_names = df_train2.keys()[:-1]

In [11]:
#print(feature_names)
#print(len(feature_names))

In [12]:
# print(y)

# u, counts = np.unique(y, return_counts=True)
# print(len(u))
# print(u)
# print(counts)

In [13]:
from imblearn.over_sampling import RandomOverSampler

In [14]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
print(X_resampled.shape, y_resampled.shape)

u, counts = np.unique(y_resampled, return_counts=True)
print(len(u))
print(u)
print(counts)

(915, 26) (915,)
15
[ 5 10 15 17 20 25 30 35 40 45 50 52 60 67 75]
[61 61 61 61 61 61 61 61 61 61 61 61 61 61 61]


In [15]:
from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_resampled, y_resampled, test_size=0.3, shuffle=True, random_state=1)

In [16]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(640, 26) (640,)
(275, 26) (275,)


In [17]:
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_names)

In [18]:
print(dtrain.feature_names)

['category', 'created_at', 'updated_at', 'char_count_sum', 'new_line_count_sum', 'talk_char_count_sum', 'new_line_count_rate_sum', 'talk_char_count_rate_sum', 'char_count_avg', 'new_line_count_avg', 'talk_char_count_avg', 'new_line_count_rate_avg', 'talk_char_count_rate_avg', 'wc_rate_0', 'wc_rate_1', 'wc_rate_2', 'wc_rate_3', 'wc_rate_4', 'wc_rate_5', 'wc_rate_6', 'wc_rate_7', 'wc_rate_8', 'wc_rate_9', 'wc_rate_10', 'wc_rate_11', 'wc_rate_12']


In [19]:
# xgb_params = {
#     'objective': 'reg:linear',
#     'eval_metric': 'rmse',
# }

# evals = [(dtrain, 'train'), (dval, 'eval')]
# evals_result = {}

# bst = xgb.train(xgb_params,
#                 dtrain,
#                 num_boost_round=100,
#                 early_stopping_rounds=20,
#                 evals=evals,
#                 evals_result=evals_result)

In [20]:
# y_pred = bst.predict(dval)
# mse = mean_squared_error(y_val, y_pred)
# print('RMSE:', math.sqrt(mse))

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import optuna

In [23]:
def objective(trial):
    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',

        # Booster Params
        #'n_estimators': trial.suggest_int('n_estimators', 0, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
    }

    evals = [(dtrain, 'train'), (dval, 'eval')]
    evals_result = {}

    bst = xgb.train(xgb_params,
                    dtrain,
                    #num_boost_round=100,
                    early_stopping_rounds=20,
                    evals=evals,
                    evals_result=evals_result,
                    verbose_eval=False)

    y_pred = bst.predict(dval)
    mse = mean_squared_error(y_val, y_pred)
    rmse = math.sqrt(mse)

    return rmse

In [27]:
#study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=10, timeout=600)
study = optuna.create_study()
#study.optimize(objective, timeout=600)
study.optimize(objective, n_trials=2000)

print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

[I 2020-08-28 19:35:53,904] Trial 0 finished with value: 10.102489731209367 and parameters: {'max_depth': 11, 'min_child_weight': 20, 'subsample': 0.5, 'colsample_bytree': 0.9}. Best is trial 0 with value: 10.102489731209367.
[I 2020-08-28 19:35:53,916] Trial 1 finished with value: 13.237795316861517 and parameters: {'max_depth': 2, 'min_child_weight': 8, 'subsample': 0.5, 'colsample_bytree': 0.5}. Best is trial 0 with value: 10.102489731209367.
[I 2020-08-28 19:35:53,945] Trial 2 finished with value: 5.635731212537364 and parameters: {'max_depth': 20, 'min_child_weight': 2, 'subsample': 0.8, 'colsample_bytree': 0.8}. Best is trial 2 with value: 5.635731212537364.
[I 2020-08-28 19:35:53,967] Trial 3 finished with value: 8.55399459844542 and parameters: {'max_depth': 11, 'min_child_weight': 12, 'subsample': 0.6, 'colsample_bytree': 0.9}. Best is trial 2 with value: 5.635731212537364.
[I 2020-08-28 19:35:53,984] Trial 4 finished with value: 6.016931648672125 and parameters: {'max_depth':

[I 2020-08-28 19:35:55,120] Trial 36 finished with value: 6.898481828148598 and parameters: {'max_depth': 6, 'min_child_weight': 2, 'subsample': 0.8, 'colsample_bytree': 0.9}. Best is trial 13 with value: 5.529830823889777.
[I 2020-08-28 19:35:55,155] Trial 37 finished with value: 5.771788605432413 and parameters: {'max_depth': 17, 'min_child_weight': 4, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 13 with value: 5.529830823889777.
[I 2020-08-28 19:35:55,182] Trial 38 finished with value: 16.201906967126437 and parameters: {'max_depth': 1, 'min_child_weight': 8, 'subsample': 0.7, 'colsample_bytree': 0.5}. Best is trial 13 with value: 5.529830823889777.
[I 2020-08-28 19:35:55,210] Trial 39 finished with value: 9.289984824210334 and parameters: {'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.8, 'colsample_bytree': 0.8}. Best is trial 13 with value: 5.529830823889777.
[I 2020-08-28 19:35:55,243] Trial 40 finished with value: 7.758196334140151 and parameters: {'max_dept

[I 2020-08-28 19:35:56,357] Trial 72 finished with value: 5.35228548119351 and parameters: {'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 59 with value: 5.103081472694784.
[I 2020-08-28 19:35:56,391] Trial 73 finished with value: 5.203586960130416 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 59 with value: 5.103081472694784.
[I 2020-08-28 19:35:56,424] Trial 74 finished with value: 5.203586960130416 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 59 with value: 5.103081472694784.
[I 2020-08-28 19:35:56,485] Trial 75 finished with value: 5.203586960130416 and parameters: {'max_depth': 10, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 59 with value: 5.103081472694784.
[I 2020-08-28 19:35:56,521] Trial 76 finished with value: 5.203586960130416 and parameters: {'max_dep

[I 2020-08-28 19:35:57,694] Trial 108 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:57,733] Trial 109 finished with value: 4.804656441178185 and parameters: {'max_depth': 13, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:57,767] Trial 110 finished with value: 5.307431382626845 and parameters: {'max_depth': 9, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:57,804] Trial 111 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:57,839] Trial 112 finished with value: 5.493275869956727 and parameters: {'max_dept

[I 2020-08-28 19:35:59,578] Trial 144 finished with value: 4.798179433245005 and parameters: {'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:59,670] Trial 145 finished with value: 5.35228548119351 and parameters: {'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:59,746] Trial 146 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:59,835] Trial 147 finished with value: 4.798179433245005 and parameters: {'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:35:59,912] Trial 148 finished with value: 5.35228548119351 and parameters: {'max_dept

...

[I 2020-08-28 19:38:11,952] Trial 1872 finished with value: 5.493275869956727 and parameters: {'max_depth': 11, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:12,024] Trial 1873 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:12,099] Trial 1874 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:12,177] Trial 1875 finished with value: 5.276312677194894 and parameters: {'max_depth': 10, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:12,273] Trial 1876 finished with value: 5.44098881058716 and parameters: {'max

[I 2020-08-28 19:38:14,861] Trial 1908 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:14,939] Trial 1909 finished with value: 5.011973913111891 and parameters: {'max_depth': 11, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:15,017] Trial 1910 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:15,094] Trial 1911 finished with value: 4.798179433245005 and parameters: {'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:15,171] Trial 1912 finished with value: 5.493275869956727 and parameters: {'ma

[I 2020-08-28 19:38:17,840] Trial 1944 finished with value: 5.103081472694784 and parameters: {'max_depth': 10, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:17,919] Trial 1945 finished with value: 5.44098881058716 and parameters: {'max_depth': 12, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:17,997] Trial 1946 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:18,076] Trial 1947 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:18,158] Trial 1948 finished with value: 4.72365553084053 and parameters: {'max_

[I 2020-08-28 19:38:21,023] Trial 1980 finished with value: 4.798179433245005 and parameters: {'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:21,101] Trial 1981 finished with value: 4.72365553084053 and parameters: {'max_depth': 11, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:21,187] Trial 1982 finished with value: 5.493275869956727 and parameters: {'max_depth': 11, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:21,266] Trial 1983 finished with value: 4.798179433245005 and parameters: {'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7}. Best is trial 79 with value: 4.72365553084053.
[I 2020-08-28 19:38:21,351] Trial 1984 finished with value: 5.44098881058716 and parameters: {'ma

Number of finished trials: 2000
Best trial:
  Value: 4.72365553084053
  Params: 
    max_depth: 11
    min_child_weight: 1
    subsample: 0.9
    colsample_bytree: 0.7
